<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/script_VAEGAN_voxelsize1_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#right click shared folder IRCMS_GAN_collaborative_database and "Add shortcut to Drive" to My drive
%cd drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-treegan/

#record paths to resources
data_path = "../../Research/Peter/Tree_3D_models_obj/obj_files/"
run_path = "./"

!pip install pytorch-lightning
!pip install trimesh
!pip install wandb -q
!apt install -y xvfb
!pip install trimesh xvfbwrapper
output.clear()

/content/drive/.shortcut-targets-by-id/1ylB2p6N0qQ-G4OsBuwcZ9C0tsqVu9ww4/IRCMS_GAN_collaborative_database/Experiments/colab-treegan


#add libraries, and login to wandb

In [4]:
import io
import os
import trimesh
import numpy as np

import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

# WandB – Import the wandb library
import wandb

from xvfbwrapper import Xvfb

In [5]:
!wandb login
output.clear()

In [6]:
#id None to start a new run. For resuming run, put the id of the run below
id = None 
resume = False
if id is None:
    id = wandb.util.generate_id()
else:
    resume = True

run = wandb.init(project="tree-gan", id=id, resume="allow", dir=run_path)
print("run id: " + str(wandb.run.id))
print("run name: " + str(wandb.run.name))
wandb.watch_called = False
wandb.run.save_code = True

run id: af3f8v3g
run name: af3f8v3g


Streaming file created twice in same run: ./wandb/run-20200902_092351-af3f8v3g/wandb-events.jsonl
Streaming file created twice in same run: ./wandb/run-20200902_092351-af3f8v3g/wandb-history.jsonl


In [7]:
#keep track of hyperparams
config = wandb.config
config.dataset = "dataset-tree:full"

config.batch_size = 8
config.array_size = 32

config.z_size = 128
config.gen_num_layer_unit = [32, 32, 32, 32]
config.dis_num_layer_unit = [16, 16, 16, 16]
config.leakyReLU = False    #leakyReLU implementation still not in modelPL
config.balance_voxel_in_space = False

config.epochs = 1000
config.vae_lr = 0.0003
config.vae_encoder_layer = 1
config.vae_decoder_layer = 2
config.d_lr = 0.0003         
config.d_layer = 1
config.vae_recon_loss_factor = 0.3
config.seed = 1234
config.log_image_interval = 5
config.log_mesh_interval = 50
config.data_augmentation = True
config.num_augment_data = 4

config.vae_opt = "Adam"
config.dis_opt = "Adam"

#dataset

In [10]:
### load our package

#clone then install
# !git clone https://github.com/buganart/BUGAN repo
# !pip install -e ./repo/
# import site
# site.main()

#directly install using pip
# !pip install git+https://github.com/buganart/BUGAN.git#egg=bugan
# output.clear()


# from bugan.functions import *
# from bugan.models import VAEGAN, VAE, Discriminator, Generator

from functionsPL import *
from modelsPL import VAEGAN, VAE, Discriminator, Generator

run.tags.append("VAEGAN")
run.group = "VAEGAN"

###     load dataset
np.random.seed(config.seed)
dataModule = DataModule(config, run)
config.num_data = dataModule.size

#train

In [13]:
#set seed
torch.manual_seed(config.seed)
torch.autograd.set_detect_anomaly(True)

#render setup
vdisplay = Xvfb()
vdisplay.start()

checkpoint_path = os.path.join(wandb.run.dir, 'checkpoint.ckpt')

if resume:
    #get file from the wandb cloud
    load_checkpoint_from_cloud(checkpoint_path = 'checkpoint.ckpt')
    #restore training state completely
    trainer = pl.Trainer(max_epochs = config.epochs, checkpoint_callback = None, resume_from_checkpoint = checkpoint_path)
else:
    trainer = pl.Trainer(max_epochs = config.epochs, checkpoint_callback = None)

#model
vaegan = VAEGAN(config, trainer, save_model_path = checkpoint_path).to(device)

trainer.fit(vaegan, dataModule)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name          | Type          | Params
------------------------------------------------
0 | vae           | VAE           | 453 K 
1 | discriminator | Discriminator | 152 K 
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
Saving latest checkpoint..


1